In [1]:
# moduos
import polars as pl

In [ ]:
from typing import List
# Funciones

def get_encoding(filename_in: str, filename_out:str) -> None:
    import chardet

    with open(filename_in, 'rb') as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']

        try:
            decoded_data = raw_data.decode(encoding=encoding)
        except UnicodeDecodeError:
            # Si falla, intenta con una lista de codificaciones comunes
            common_encodings = ['utf-8', 'iso-8859-1', 'cp1252']
            for enc in common_encodings:
                try:
                    decoded_data = raw_data.decode(encoding=enc)
                    break
                except UnicodeDecodeError:
                    continue
            else:
                raise UnicodeDecodeError(f"No se pudo decodificar el archivo con las codificaciones probadas. Último intento fallido con {encoding}")

    with open(filename_out, 'w', encoding='utf-8') as f2:
        f2.write(decoded_data)

def tweak_df(df: pl.DataFrame, columns_trans: list, schema: dict) -> pl.DataFrame:
  # aplicación de funciones para el df
  # acepta lista de expresiones
  return df.with_columns(columns_trans).cast(schema)

def transformations(file_name: str,tweak_list: list,schema_dict: dict) -> pl.DataFrame:
  # decodificando
  get_encoding(f"{file_name}.csv",f"{file_name}_decoded.csv")
  # leyendo el archivo
  df = pl.read_csv(f"{file_name}_decoded.csv")
  # aplicando los cambios al df
  return tweak_df(df,tweak_list,schema_dict)

Nos metimos al IFT (instituto federal de telecomunicaciones) en especial a la sección de servicios móvil de acceso a internet, para delimitar

Subsecciones:
  - Lineas (Ya)
  - Lineas (Serie hitórica desde 2010) (Ya)
  - Tráfico de datos (TB) (Ya)
  - Índice de concentración (IHH) (Ya)
  - Participación de mercado (Serie desde 2010) (Ya)
  - Líneas por cada 100  habitantes (Estatal) (No)
  -Líneas por cada 100 habitantes (Serie historica desde 2010) (No)

## LINEAS
Tabla de información de las líneas de servicio móvil de acceso a internet por esquema de pago (prepago y pospago), serie mensual desde el 2013.

[diccionario de datos](https://docs.google.com/spreadsheets/d/176qSChhhpF43hzslsFscsAcBblJ0Wa_cyBHnzWTq1Eg/edit?gid=0#gid=0)

### To do:
- eda
- encontrar los valores distintos

### Done:
- Ir de acuerdo al esquema en el diccionario de datos



In [ ]:
#Primero vamos a ver lineas
#nos falta un catálogo de datos
!wget https://bit.ift.org.mx/descargas/datos/tabs/TD_LINEAS_INTMOVIL_ITE_VA.csv

In [ ]:
# creación de esquema de acuera al diccionario de datos
schema_lineas = {
    "FECHA": pl.Date,
    "ANIO": pl.Int16,
    "MES": pl.Int8,
    # "K_GRUPO": pl.Categorical,
    "GRUPO": pl.Categorical,
    "K_EMPRESA": pl.Categorical,
    "EMPRESA": pl.Categorical,
    "CONCESIONARIO": pl.Categorical,
    "L_PREPAGO_E": pl.Int32,
    "L_POSPAGO_E": pl.Int32,
    "L_POSPAGOC_E": pl.Int32,
    "L_POSPAGOL_E": pl.Int32,
    "L_NO_ESPECIFICADO_E": pl.Int32,
    "L_TOTAL_E": pl.Int32,
    "FOLIO": pl.Categorical
  }

tweak_columns = [
    pl.col( "L_PREPAGO_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col( "L_POSPAGO_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col( "L_POSPAGOC_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col( "L_POSPAGOL_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col( "L_NO_ESPECIFICADO_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col( "L_TOTAL_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col("FECHA").str.to_date("%d%b%Y"),
    pl.col("FOLIO").cast(pl.Utf8)
  ]

In [ ]:
df_lineas = transformations("TD_LINEAS_INTMOVIL_ITE_VA",tweak_columns,schema_lineas)

In [4]:
from airflow import DAG

In [6]:
help(DAG)

Help on class DAG in module airflow.models.dag:

class DAG(airflow.utils.log.logging_mixin.LoggingMixin)
 |  DAG(dag_id: 'str', description: 'str | None' = None, schedule: 'ScheduleArg' = <airflow.utils.types.ArgNotSet object at 0x7fd645771640>, schedule_interval: 'ScheduleIntervalArg' = <airflow.utils.types.ArgNotSet object at 0x7fd645771640>, timetable: 'Timetable | None' = None, start_date: 'datetime | None' = None, end_date: 'datetime | None' = None, full_filepath: 'str | None' = None, template_searchpath: 'str | Iterable[str] | None' = None, template_undefined: 'type[jinja2.StrictUndefined]' = <class 'jinja2.runtime.StrictUndefined'>, user_defined_macros: 'dict | None' = None, user_defined_filters: 'dict | None' = None, default_args: 'dict | None' = None, concurrency: 'int | None' = None, max_active_tasks: 'int' = 16, max_active_runs: 'int' = 16, max_consecutive_failed_dag_runs: 'int' = 0, dagrun_timeout: 'timedelta | None' = None, sla_miss_callback: 'None | SLAMissCallback | list

In [1]:
from airflow.decorators import dag, task
from airflow import tasm

In [3]:
help(task)

Help on TaskDecoratorCollection in module airflow.decorators object:

class TaskDecoratorCollection(builtins.object)
 |  Implementation to provide the ``@task`` syntax.
 |
 |  Methods defined here:
 |
 |  __getattr__(self, name: 'str') -> 'TaskDecorator'
 |      Dynamically get provider-registered task decorators, e.g. ``@task.docker``.
 |
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |
 |  __call__ = python_task(python_callable: 'Callable | None' = None, multiple_outputs: 'bool | None' = None, **kwargs) -> 'TaskDecorator' from airflow.decorators.python
 |      Wrap a function into an Airflow operator.
 |
 |      Accepts kwargs for operator kwarg. Can be reused in a single DAG.
 |
 |      :param python_callable: Function to decorate
 |      :param multiple_outputs: If set to True, the decorated function's return value will be unrolled to
 |          multiple XCom values. Dict will unroll to XCom values with its keys as XCom

In [ ]:
from airflow import DAG
from airflow.decorators import task

with DAG(dag_id="prueba",schedule=None) as dag:
    @task(task_id="primera task", task_display_name="primera task")
    def task_1():
        return "esta es la primera prueba"


In [10]:
import requests
response = requests.get("https://bit.ift.org.mx/descargas/datos/tabs/TD_LINEAS_INTMOVIL_ITE_VA.csv")

In [11]:
response.status_code

200

In [12]:
response.text

'FOLIO,FECHA,ANIO,MES,K_GRUPO,GRUPO,K_EMPRESA,EMPRESA,CONCESIONARIO,L_PREPAGO_E,L_POSPAGO_E,L_POSPAGOC_E,L_POSPAGOL_E,L_NO_ESPECIFICADO_E,L_TOTAL_E\r\n100418,15JAN2019,2019,01,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","2,962",,,,,"2,962"\r\n100418,15FEB2019,2019,02,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","25,469",,,,,"25,469"\r\n100418,15MAR2019,2019,03,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","21,501",,,,,"21,501"\r\n100418,15APR2019,2019,04,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","12,020",,,,,"12,020"\r\n100418,15MAY2019,2019,05,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","12,020",,,,,"12,020"\r\n100418,15JUN2019,2019,06,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","12,020",,,,,"12,020"\r\n100418,15JUL2019,2019,07,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.",,,"1,833",,,"1,833"\r\n100418,15AUG2019,2019,08,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CAB

In [13]:
response.encoding

'ISO-8859-1'

In [15]:
response.encoding = 'utf-8'

In [14]:
response.content

b'FOLIO,FECHA,ANIO,MES,K_GRUPO,GRUPO,K_EMPRESA,EMPRESA,CONCESIONARIO,L_PREPAGO_E,L_POSPAGO_E,L_POSPAGOC_E,L_POSPAGOL_E,L_NO_ESPECIFICADO_E,L_TOTAL_E\r\n100418,15JAN2019,2019,01,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","2,962",,,,,"2,962"\r\n100418,15FEB2019,2019,02,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","25,469",,,,,"25,469"\r\n100418,15MAR2019,2019,03,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","21,501",,,,,"21,501"\r\n100418,15APR2019,2019,04,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","12,020",,,,,"12,020"\r\n100418,15MAY2019,2019,05,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","12,020",,,,,"12,020"\r\n100418,15JUN2019,2019,06,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","12,020",,,,,"12,020"\r\n100418,15JUL2019,2019,07,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.",,,"1,833",,,"1,833"\r\n100418,15AUG2019,2019,08,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CA

In [16]:
response.encoding

'utf-8'

In [17]:
response.text

'FOLIO,FECHA,ANIO,MES,K_GRUPO,GRUPO,K_EMPRESA,EMPRESA,CONCESIONARIO,L_PREPAGO_E,L_POSPAGO_E,L_POSPAGOC_E,L_POSPAGOL_E,L_NO_ESPECIFICADO_E,L_TOTAL_E\r\n100418,15JAN2019,2019,01,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","2,962",,,,,"2,962"\r\n100418,15FEB2019,2019,02,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","25,469",,,,,"25,469"\r\n100418,15MAR2019,2019,03,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","21,501",,,,,"21,501"\r\n100418,15APR2019,2019,04,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","12,020",,,,,"12,020"\r\n100418,15MAY2019,2019,05,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","12,020",,,,,"12,020"\r\n100418,15JUN2019,2019,06,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.","12,020",,,,,"12,020"\r\n100418,15JUL2019,2019,07,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CABLE, S.A. DE C.V.",,,"1,833",,,"1,833"\r\n100418,15AUG2019,2019,08,G008,MEGACABLE-MCM,C0378,MEGACABLE,"MEGA CAB

In [36]:
import polars as pl
from io import StringIO
import requests

def download_files(table_name: str) -> pl.DataFrame:
    url = f"https://bit.ift.org.mx/descargas/datos/tabs/{table_name}.csv"
    try:
        response = requests.get(url)
        response.raise_for_status()
        
    except requests.exceptions.RequestException as e:
        print(f"Error inesperado: {e}")
    
    response.encoding = 'utf-8'
    csv_string = StringIO(response.text)
    
    return pl.read_csv(csv_string)
    
def tweak_df(df: pl.DataFrame, columns_transformations: list, schema: dict) -> pl.DataFrame:
  # aplicación de funciones para el df
  # acepta lista de expresiones
  return df.with_columns(columns_transformations).cast(schema)



In [55]:
# creación de esquema de acuera al diccionario de datos
schema_lineas = {
    "FECHA": pl.Date,
    "ANIO": pl.Int16,
    "MES": pl.Int8,
    # "K_GRUPO": pl.Categorical,
    #"GRUPO": pl.Categorical,
    #"K_EMPRESA": pl.Categorical,
    #"EMPRESA": pl.Categorical,
    #"CONCESIONARIO": pl.Categorical,
    "L_PREPAGO_E": pl.Int32,
    "L_POSPAGO_E": pl.Int32,
    "L_POSPAGOC_E": pl.Int32,
    "L_POSPAGOL_E": pl.Int32,
    "L_NO_ESPECIFICADO_E": pl.Int32,
    "L_TOTAL_E": pl.Int32,
    #"FOLIO": pl.Categorical
  }

tweak_columns = [
    pl.col( "L_PREPAGO_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col( "L_POSPAGO_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col( "L_POSPAGOC_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col( "L_POSPAGOL_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col( "L_NO_ESPECIFICADO_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col( "L_TOTAL_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col("FECHA").str.to_date("%d%b%Y"),
    pl.col("FOLIO").cast(pl.Utf8)
  ]
diccionario_datos = {
    "Tabla_Lineas":{
        "schema":{
            "FECHA": pl.Date,
            "ANIO": pl.Int16,
            "MES": pl.Int8,
            # "K_GRUPO": pl.Categorical,
            #"GRUPO": pl.Categorical,
            #"K_EMPRESA": pl.Categorical,
            #"EMPRESA": pl.Categorical,
            #"CONCESIONARIO": pl.Categorical,
            "L_PREPAGO_E": pl.Int32,
            "L_POSPAGO_E": pl.Int32,
            "L_POSPAGOC_E": pl.Int32,
            "L_POSPAGOL_E": pl.Int32,
            "L_NO_ESPECIFICADO_E": pl.Int32,
            "L_TOTAL_E": pl.Int32,
            #"FOLIO": pl.Categorical
          },
         "tweak_columns":[
            pl.col( "L_PREPAGO_E" ).str.replace_all(",","").str.to_integer(base=10),
            pl.col( "L_POSPAGO_E" ).str.replace_all(",","").str.to_integer(base=10),
            pl.col( "L_POSPAGOC_E" ).str.replace_all(",","").str.to_integer(base=10),
            pl.col( "L_POSPAGOL_E" ).str.replace_all(",","").str.to_integer(base=10),
            pl.col( "L_NO_ESPECIFICADO_E" ).str.replace_all(",","").str.to_integer(base=10),
            pl.col( "L_TOTAL_E" ).str.replace_all(",","").str.to_integer(base=10),
            pl.col("FECHA").str.to_date("%d%b%Y"),
            pl.col("FOLIO").cast(pl.Utf8)
          ]
    },
    "Tabla_TB":{
        "schema":{
            "ANIO":pl.Int16,
            "MES":pl.Int8,
            "FECHA":pl.Date,
            "FOLIO":pl.Categorical,
            # "K_GRUPO":pl.Categorical,
            "GRUPO":pl.Categorical,
            "K_EMPRESA":pl.Categorical,
            "EMPRESA":pl.Categorical,
            "CONCESIONARIO":pl.Categorical,
            "TRAF_TB_2G_E":pl.Float32,
            "TRAF_TB_3G_E":pl.Float32,
            "TRAF_TB_4G_E":pl.Float32,
            "TRAF_TB_NO_ESPECIFICADO_E":pl.Float32,
            "TRAF_TB_E":pl.Float32
        },
        "tweak_columns":[
            pl.col('TRAF_TB_2G_E').cast(pl.Float32),
            pl.col('TRAF_TB_3G_E').str.replace_all(",","").cast(pl.Float32),
            pl.col('TRAF_TB_4G_E').str.replace_all(",","").cast(pl.Float32),
            pl.col('TRAF_TB_NO_ESPECIFICADO_E').str.replace_all(",","").cast(pl.Float32),
            pl.col('TRAF_TB_E').str.replace_all(",","").cast(pl.Float32),
            pl.col('FOLIO').cast(pl.Utf8),
            pl.coalesce(pl.col('FECHA').str.to_date(format="%d%b%Y",strict=False),pl.col('FECHA').str.to_date(format="%d-%b-%y",strict=False))
            ]
    },
    "Tabla_IHH":{
        "schema":{
            "ANIO":pl.Int16,
            "MES":pl.Int8,
            "IHH_INTMOVIL_E":pl.Int16
        },
        "tweak_columns": [
            pl.col('IHH_INTMOVIL_E').str.replace_all(",",""),
            pl.col('FECHA').str.to_date(format="%d%b%Y")
        ]
    },
    "Tabla_Participación_de_Mercado":{
        "schema": {
            "ANIO":pl.Int16,
            "MES":pl.Int8,
            # "K_GRUPO":pl.Categorical,
            "GRUPO":pl.Categorical
        },
        "tweak_columns":[
            pl.col('MARKET_SHARE').str.replace_all("%","").cast(pl.Float32),
            pl.col('FECHA').str.to_date(format="%d%b%Y")
        ]

    }
}

In [51]:
prueba['tabla2']['tweak_columns'][0]

<Expr ['col("TRAF_TB_2G_E").strict_cas…'] at 0x7F38EE850320>

In [54]:
prueba['Tabla_IHH']['schema']

{'ANIO': Int16, 'MES': Int8, 'IHH_INTMOVIL_E': Int16}

In [42]:
import json 

col_string = [str(item) for item in tweak_columns]
col_string

['col("L_PREPAGO_E").str.replace([String(,), String()]).str.to_integer([dyn int: 10])',
 'col("L_POSPAGO_E").str.replace([String(,), String()]).str.to_integer([dyn int: 10])',
 'col("L_POSPAGOC_E").str.replace([String(,), String()]).str.to_integer([dyn int: 10])',
 'col("L_POSPAGOL_E").str.replace([String(,), String()]).str.to_integer([dyn int: 10])',
 'col("L_NO_ESPECIFICADO_E").str.replace([String(,), String()]).str.to_integer([dyn int: 10])',
 'col("L_TOTAL_E").str.replace([String(,), String()]).str.to_integer([dyn int: 10])',
 'col("FECHA").str.strptime([String(raise)])',
 'col("FOLIO").strict_cast(String)']

In [59]:
nombre = "TD_LINEAS_INTMOVIL_ITE_VA"
prueba = download_files(nombre)
prueba_tweaked = tweak_df(prueba,diccionario_datos['Tabla_Lineas']['tweak_columns'],diccionario_datos['Tabla_Lineas']['schema'])
print(prueba_tweaked)

shape: (2_297, 15)
┌────────┬────────────┬──────┬─────┬───┬──────────────┬──────────────┬─────────────────┬───────────┐
│ FOLIO  ┆ FECHA      ┆ ANIO ┆ MES ┆ … ┆ L_POSPAGOC_E ┆ L_POSPAGOL_E ┆ L_NO_ESPECIFICA ┆ L_TOTAL_E │
│ ---    ┆ ---        ┆ ---  ┆ --- ┆   ┆ ---          ┆ ---          ┆ DO_E            ┆ ---       │
│ str    ┆ date       ┆ i16  ┆ i8  ┆   ┆ i32          ┆ i32          ┆ ---             ┆ i32       │
│        ┆            ┆      ┆     ┆   ┆              ┆              ┆ i32             ┆           │
╞════════╪════════════╪══════╪═════╪═══╪══════════════╪══════════════╪═════════════════╪═══════════╡
│ 100418 ┆ 2019-01-15 ┆ 2019 ┆ 1   ┆ … ┆ null         ┆ null         ┆ null            ┆ 2962      │
│ 100418 ┆ 2019-02-15 ┆ 2019 ┆ 2   ┆ … ┆ null         ┆ null         ┆ null            ┆ 25469     │
│ 100418 ┆ 2019-03-15 ┆ 2019 ┆ 3   ┆ … ┆ null         ┆ null         ┆ null            ┆ 21501     │
│ 100418 ┆ 2019-04-15 ┆ 2019 ┆ 4   ┆ … ┆ null         ┆ null         ┆ n